#### Импорты

In [1]:
import momepy
import numpy as np
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
from metrics.indicators import get_intermodal
from metrics.grade_territory import grade_territory
from graph_builder.graphbuilder import get_graph_from_polygon, get_frame, assign_city_names_to_nodes
from graph_builder.availability_estimation import availability_matrix, find_median
from graph_builder.criteria import weight_territory, assign_grades

/Users/test/Library/Caches/pypoetry/virtualenvs/transport-frames-QNnroJIm-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Загрузка графа

In [9]:
lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
russia = ox.geocode_to_gdf("Russia")

citygraph = get_graph_from_polygon(city, crs=32636,country_polygon=russia)

d:\IT\GitHub\transport_frames\.venv\Lib\site-packages\osmnx\_overpass.py:254: UserWarning: This area is 52 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


#### Выделение каркаса

In [12]:
regions = gpd.read_file('data/russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

frame = get_frame(citygraph, regions, city)

C:\Users\romab\AppData\Local\Temp\ipykernel_14268\4137021309.py:10: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  frame = get_frame(citygraph, regions, city)
d:\IT\GitHub\transport_frames\transport_frames\graph_builder\graphbuilder.py:463: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  n.loc[(n['nodeID']==path[j]),'weight']+= weight
d:\IT\GitHub\transport_frames\transport_frames\graph_builder\graphbuilder.py:464: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  e.loc[(e['node_start'] == path[j]) & (e['node_end'] == path[j+1]), 'weight'] +=weight


#### Перенос точек на граф

In [13]:
cities = gpd.read_file('data/18_centers.geojson')
frame = assign_city_names_to_nodes(cities.to_crs(frame.graph['crs']), momepy.nx_to_gdf(frame)[0], frame, name_col='name18', max_distance=1200)

#### Оценка территории

In [15]:
p1 = gpd.read_file('data/Аэродром лодейнопольское поселение.geojson')
p2 = gpd.read_file('data/Аэродром Сиверск .geojson')
p3 = gpd.read_file('data/project Светогорского поселения.geojson')
p4 = gpd.read_file('data/project Шлиссельбург.geojson')
neudobiya = pd.concat([p1,p2,p3,p4])

In [16]:
n_grade = grade_territory(neudobiya, frame, include_priority=True)

#### Расчет критериев оценки состояния территории

In [ ]:
polygons188 = gpd.read_file("data/polygons188.geojson")
points = gpd.read_file("data/points.geojson")

p1 = gpd.read_file('data/geojson/Аэродром лодейнопольское поселение.geojson')
p2 = gpd.read_file('data/geojson/Аэродром Сиверск .geojson')
p3 = gpd.read_file('data/geojson/project Светогорского поселения.geojson')
p4 = gpd.read_file('data/geojson/project Шлиссельбург.geojson')

ferry = gpd.read_file('data/geojson/water_transport_LO.geojson')
aero = gpd.read_file('data/geojson/airports_local_LO.geojson')
r_stops = gpd.read_file('data/geojson/ЖД остановки.geojson')
fuel = gpd.read_file('data/geojson/fuel.geojson')
b_stops = gpd.read_file('data/geojson/Остановки ЛО.geojson')

In [ ]:
inter = get_intermodal(city_id=, crs=32636)

In [ ]:
adj_mx = availability_matrix(citygraph, points)
p = find_median(points,adj_mx, polygon)

In [ ]:
p_agg = p[p['to_service'] < np.finfo(np.float64).max].copy()
res = gpd.sjoin(p_agg, polygons188, how='left', op='within').groupby('index_right').median(['to_service']).reset_index()
result_df = pd.merge(polygons188.reset_index(), res, left_on='index', right_on='index_right', how='left').drop(columns=['fid_right']).rename(columns={'to_service': 'in_car'})
result_df = result_df.drop(columns=['index_right'])

In [ ]:
adj_mx_inter = availability_matrix(inter,points,graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK])
p_inter = find_median(points, adj_mx_inter, result_df)
points_inter = p_inter[p_inter['to_service'] < np.finfo(np.float64).max].copy()

In [ ]:
res_inter = gpd.sjoin(points_inter, polygons188, how="left", predicate="within").groupby('index_right').median(['to_service']).reset_index()
result_df_inter = pd.merge(result_df, res_inter, left_on='index', right_on='index_right', how='left').drop(columns=['index_right', 'fid_right']).rename(columns={'to_service': 'in_inter'})

In [ ]:
# Grade the territories
graded_gdf = weight_territory(n_grade, r_stops, b_stops, ferry, aero)

In [ ]:
result = assign_grades(graded_gdf[['name', 'geometry', 'grade', 'weight']], result_df_inter[['index', 'fid', 'name', 'geometry', 'in_car', 'in_inter']])
result.to_file('data/geojson/result_assesment.geojson', driver='GeoJSON')